In [6]:
import os
import pandas as pd
import json

In [3]:
%run utils/__init__.py

In [2]:
%run metrics/__init__.py

In [37]:
results_folder = os.path.join(WORKSPACE_DIR, 'classification', 'results')
run_names = os.listdir(results_folder)
run_names.remove('debug')
run_names

['0704_232547_covid-kaggle_resnet-50_lr1e-06',
 '0704_062601_covid-kaggle_densenet-121_lr1e-06',
 '0704_015027_covid-kaggle_densenet-121_lr1e-06_os',
 '0705_013805_covid-kaggle_resnet-50_lr1e-06_noig_os',
 '0704_161918_covid-kaggle_densenet-121_lr1e-06_noig',
 '0705_055411_covid-kaggle_resnet-50_lr1e-06_noig',
 '0704_190958_covid-kaggle_resnet-50_lr1e-06_os',
 '0704_114753_covid-kaggle_densenet-121_lr1e-06_noig_os']

In [38]:
all_results = []

for run_name in run_names:
    filepath = os.path.join(results_folder, run_name, 'metrics.json')
    
    if not os.path.isfile(filepath):
        print('Not found for: ', run_name)
        continue

    with open(filepath, 'r') as f:
        results = json.load(f)

    all_results.append((run_name, results))

len(all_results)

8

In [39]:
def create_results_df(key):
    run_names = [run_name for run_name, _ in all_results]
    
    results_dicts = [d[key] for _, d in all_results]
    
    df = pd.DataFrame(results_dicts, index=run_names)
    return df

In [42]:
results_df_val = create_results_df('val')
results_df_test = create_results_df('test')

In [45]:
results_df_val.sort_values(['recall_covid', 'prec_covid'], ascending=False)

,loss,acc,prec_covid,prec_pneumonia,prec_normal,recall_covid,recall_pneumonia,recall_normal,spec_covid,spec_pneumonia,spec_normal,cm
0704_015027_covid-kaggle_densenet-121_lr1e-06_os,0.074927,0.951557,0.875000,0.983871,0.936170,1.000000,0.910448,0.985075,0.988806,0.987097,0.941935,"[[21, 0, 0], [3, 123, 8], [0, 3, 131]]"
0704_190958_covid-kaggle_resnet-50_lr1e-06_os,0.099760,0.955017,1.000000,0.976378,0.929577,0.952381,0.925373,0.985075,1.000000,0.980645,0.935484,"[[20, 1, 0], [0, 124, 10], [0, 3, 131]]"
0704_114753_covid-kaggle_densenet-121_lr1e-06_noig_os,0.104679,0.927336,0.740741,0.975410,0.921429,0.952381,0.888060,0.962687,0.973881,0.980645,0.929032,"[[20, 0, 1], [3, 121, 10], [2, 3, 129]]"
0705_013805_covid-kaggle_resnet-50_lr1e-06_noig_os,1.833148,0.709343,0.301887,0.786408,0.812030,0.761905,0.604478,0.805970,0.861940,0.858065,0.838710,"[[15, 2, 4], [28, 90, 16], [5, 22, 107]]"
0704_232547_covid-kaggle_resnet-50_lr1e-06,0.944305,0.878893,0.777778,0.949580,0.832298,0.333333,0.843284,1.000000,0.992537,0.961290,0.825806,"[[5, 8, 8], [1, 114, 19], [0, 1, 133]]"
0704_161918_covid-kaggle_densenet-121_lr1e-06_noig,4.084516,0.757785,0.666667,0.894737,0.691099,0.095238,0.634328,0.985075,0.996269,0.935484,0.619355,"[[1, 10, 10], [0, 94, 40], [0, 4, 130]]"
0705_055411_covid-kaggle_resnet-50_lr1e-06_noig,6.326619,0.730104,0.666667,0.720280,0.741259,0.095238,0.768657,0.791045,0.996269,0.741935,0.761290,"[[2, 13, 6], [0, 106, 28], [0, 31, 103]]"
0704_062601_covid-kaggle_densenet-121_lr1e-06,3.724915,0.754325,0.333333,0.878788,0.695187,0.047619,0.649254,0.970149,0.992537,0.922581,0.632258,"[[1, 9, 11], [1, 95, 38], [1, 4, 129]]"


In [44]:
results_df_test.sort_values(['recall_covid', 'prec_covid'], ascending=False)

,loss,acc,prec_covid,prec_pneumonia,prec_normal,recall_covid,recall_pneumonia,recall_normal,spec_covid,spec_pneumonia,spec_normal,cm
0704_015027_covid-kaggle_densenet-121_lr1e-06_os,0.008223,0.965398,1.000000,0.984375,0.942857,1.000000,0.940299,0.985075,1.000000,0.987097,0.948387,"[[21, 0, 0], [0, 127, 7], [0, 2, 132]]"
0704_190958_covid-kaggle_resnet-50_lr1e-06_os,0.002729,0.975779,0.875000,0.992366,0.977612,1.000000,0.970149,0.977612,0.988806,0.993548,0.980645,"[[21, 0, 0], [1, 131, 2], [2, 2, 130]]"
0704_114753_covid-kaggle_densenet-121_lr1e-06_noig_os,0.067778,0.951557,0.724138,0.984127,0.970149,1.000000,0.925373,0.970149,0.970149,0.987097,0.974194,"[[19, 2, 0], [5, 125, 4], [2, 2, 130]]"
0705_013805_covid-kaggle_resnet-50_lr1e-06_noig_os,0.162184,0.692042,0.287879,0.788462,0.831933,0.904762,0.611940,0.738806,0.824627,0.858065,0.870968,"[[18, 2, 1], [30, 88, 16], [11, 25, 98]]"
0704_232547_covid-kaggle_resnet-50_lr1e-06,1.058202,0.903114,0.909091,0.975000,0.848101,0.476190,0.873134,1.000000,0.996269,0.980645,0.845161,"[[7, 5, 9], [1, 120, 13], [0, 0, 134]]"
0704_062601_covid-kaggle_densenet-121_lr1e-06,2.662862,0.757785,0.800000,0.870968,0.701571,0.190476,0.604478,1.000000,0.996269,0.922581,0.632258,"[[4, 13, 4], [0, 92, 42], [0, 0, 134]]"
0704_161918_covid-kaggle_densenet-121_lr1e-06_noig,3.421237,0.737024,0.333333,0.879121,0.676923,0.047619,0.597015,0.985075,0.992537,0.929032,0.593548,"[[0, 12, 9], [0, 87, 47], [0, 2, 132]]"
0705_055411_covid-kaggle_resnet-50_lr1e-06_noig,5.772729,0.733564,0.000000,0.751938,0.718750,0.000000,0.723881,0.858209,1.000000,0.793548,0.709677,"[[0, 16, 5], [0, 100, 34], [0, 28, 106]]"
